<a href="https://colab.research.google.com/github/cryptoshepherd/Crypto-Price-Prediction-App/blob/main/ETH_Price_Predition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

     |████████████████████████████████| 67.3 MB 73 kB/s 
     |████████████████████████████████| 64 kB 1.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1 requires cmdstanpy>=1.0.1, but you have cmdstanpy 0.9.5 which is incompatible.
     |████████████████████████████████| 62 kB 1.1 MB/s 


In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD', start_date, today)
eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-04,"$1,556.90","$1,578.01","$1,543.70","$1,577.64","$1,577.64",8884144998
2022-09-05,"$1,577.88","$1,621.66","$1,559.78","$1,617.18","$1,617.18",13060541168
2022-09-06,"$1,617.24","$1,680.60","$1,561.75","$1,561.75","$1,561.75",22946059125
2022-09-07,"$1,560.91","$1,651.05","$1,500.01","$1,629.91","$1,629.91",19560363854
2022-09-08,"$1,629.81","$1,655.06","$1,603.06","$1,635.35","$1,635.35",17621046717


In [ ]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1765 entries, 2017-11-09 to 2022-09-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1765 non-null   float64
 1   High       1765 non-null   float64
 2   Low        1765 non-null   float64
 3   Close      1765 non-null   float64
 4   Adj Close  1765 non-null   float64
 5   Volume     1765 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 96.5 KB


In [ ]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
eth_df.reset_index(inplace=True)

In [ ]:
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
df = eth_df[['Date', 'Open']]
df.tail()

,Date,Open
1760,2022-09-04,"$1,556.90"
1761,2022-09-05,"$1,577.88"
1762,2022-09-06,"$1,617.24"
1763,2022-09-07,"$1,560.91"
1764,2022-09-08,"$1,629.81"


In [ ]:
new_names = {
    "Date": "ds",
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [ ]:
df.tail()

,ds,y
1760,2022-09-04,"$1,556.90"
1761,2022-09-05,"$1,577.88"
1762,2022-09-06,"$1,617.24"
1763,2022-09-07,"$1,560.91"
1764,2022-09-08,"$1,629.81"


In [ ]:
x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [ ]:
m = Prophet(seasonality_mode='multiplicative')
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future = m.make_future_dataframe(periods=30)

In [ ]:
future.tail()

,ds
1790,2022-10-04
1791,2022-10-05
1792,2022-10-06
1793,2022-10-07
1794,2022-10-08


In [ ]:
forecast = m.predict(future)

In [ ]:
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
1790,2022-10-04,"$1,531.50","$1,109.58","$1,573.16","$1,526.22","$1,541.49",$-0.13,$-0.13,$-0.13,$-0.00,$-0.00,$-0.00,$-0.13,$-0.13,$-0.13,$0.00,$0.00,$0.00,"$1,334.51"
1791,2022-10-05,"$1,525.68","$1,109.62","$1,575.75","$1,519.02","$1,536.32",$-0.12,$-0.12,$-0.12,$-0.00,$-0.00,$-0.00,$-0.12,$-0.12,$-0.12,$0.00,$0.00,$0.00,"$1,342.19"
1792,2022-10-06,"$1,519.86","$1,112.91","$1,599.83","$1,512.20","$1,531.75",$-0.10,$-0.10,$-0.10,$0.01,$0.01,$0.01,$-0.11,$-0.11,$-0.11,$0.00,$0.00,$0.00,"$1,360.47"
1793,2022-10-07,"$1,514.03","$1,131.83","$1,614.99","$1,505.30","$1,527.31",$-0.10,$-0.10,$-0.10,$0.00,$0.00,$0.00,$-0.10,$-0.10,$-0.10,$0.00,$0.00,$0.00,"$1,364.14"
1794,2022-10-08,"$1,508.21","$1,128.79","$1,596.97","$1,498.24","$1,522.54",$-0.09,$-0.09,$-0.09,$-0.01,$-0.01,$-0.01,$-0.09,$-0.09,$-0.09,$0.00,$0.00,$0.00,"$1,365.12"


In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1790,2022-10-04,"$1,334.51","$1,109.58","$1,573.16"
1791,2022-10-05,"$1,342.19","$1,109.62","$1,575.75"
1792,2022-10-06,"$1,360.47","$1,112.91","$1,599.83"
1793,2022-10-07,"$1,364.14","$1,131.83","$1,614.99"
1794,2022-10-08,"$1,365.12","$1,128.79","$1,596.97"


In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

1541.6246141053189

In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)